In [1]:
import numpy as np
from scipy import stats

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
def calculate_result(val_player, val_opponent, action_player, action_opponent):
    '''Function that calsulates auction result'''
    
    bid_player   = val_player*action_player
    bid_opponent = val_opponent*action_opponent
    
    win_player = (bid_player>bid_opponent)*(val_player-bid_player)
    
    return win_player

def prepare_data(n, model=None, start=False, n_bids=100):
    '''Prepare X and y for the neural network'''
    
    # valuations and actions of the player
    val_player   = np.random.beta(1,1,n)
    action_player  = np.random.beta(1,1,n)
    val_opponent = np.random.beta(1,1,n)
    
    # if first iteration - opponent plays randomly
    if start==True:
        action_opponent = np.random.beta(1,1,n)
        opponent = np.concatenate((val_opponent.reshape(n,1), action_opponent.reshape(n,1)), axis=1)
    else:
        # opponent chooses optimal bid given his valuation
        val_opponent = np.repeat(val_opponent, n_bids)
        action_opponent = np.tile(np.linspace(start=0, stop=1, num=n_bids), n)
        
        # gather private values and bids together
        opponent = np.concatenate((val_opponent.reshape(n*n_bids,1), action_opponent.reshape(n*n_bids,1)), axis=1)
        
        # determine optimal bid
        opponent = opt_bid(model, opponent)
    
    # outcome of the auction
    y = calculate_result(val_player, opponent[:,0], action_player, opponent[:,1])
    
    X = np.concatenate((val_player.reshape(n,1), action_player.reshape(n,1)), axis=1)
    y = np.reshape(y ,[n,]) 
    
    return X, y

def fit_model(X, y):
    
    # create a neural network
    mlp = MLPRegressor(alpha=0.001, hidden_layer_sizes = (10,5,3), max_iter = 1000, 
                   activation = 'logistic', verbose = True, learning_rate = 'adaptive')
    # fit model
    mlp.fit(X, y)
    
    return mlp

def opt_bid(model, X):
    '''Find bid with the highest expected profit'''
    
    # predict expected profit for each pair
    cand_predict = model.predict(X)
    cand_predict[cand_predict <= 0] = 0
    cand_predict[cand_predict >= 1] = 1

    # add expected profit to cand
    cand_res = np.concatenate((X, cand_predict.reshape(X.shape[0],1)), axis=1)

    # sort cand_res in descending order by the expected profit
    cand_res = cand_res[cand_res[:,2].argsort()[::-1],:]

    # keep only best bids
    unique_keys, indices = np.unique(cand_res[:,0], return_index=True)
    cand_res = cand_res[indices,:]
    
    
    return cand_res

def track_progress(model):
    n_val = 5
    n_bids = 100
    n_total = n_val*n_bids

    # create private values and bids
    val_player = np.repeat([0.1, 0.3, 0.5, 0.7, 0.9], n_bids)
    action_player = np.tile(np.linspace(start=0, stop=1, num=n_bids), n_val)

    # gather private values and bids together
    cand = np.concatenate((val_player.reshape(n_total,1), action_player.reshape(n_total,1)), axis=1)
    
    return opt_bid(model, cand)

### 20 iterations - uniform distribution of bits

In [3]:
np.random.seed(4)

# number of observations in the dataset
n = 1000000

vector_opt_bids = np.array([0.1, 0.3, 0.5, 0.7, 0.9]).reshape(5,1)

for iter in range(20):
    
    print("Iteration #", iter, "\n")
    
    # prepare data
    if iter==0:
        X, y = prepare_data(n, start=True)
    else:
        X, y = prepare_data(n, model=mlp)
        # save this model
        mlp_prev = mlp
        
    # fit the model
    mlp = fit_model(X, y)
    
    # print optimal bids
    vector_opt_bids = np.concatenate((vector_opt_bids, track_progress(mlp)[:,1].reshape(5,1)), axis=1)
    print(vector_opt_bids[:,[0,-1]])
    
    # if there are extreme values in vector_opt_bids - repeat the iteration
    if (np.sum(vector_opt_bids[:,-1]==0) + np.sum(vector_opt_bids[:,-1]==1))!=0:
        print("Repeating iteration")
        mlp = mlp_prev

    del X,y

Iteration # 0 

Iteration 1, loss = 0.01691083
Iteration 2, loss = 0.01237798
Iteration 3, loss = 0.01155421
Iteration 4, loss = 0.01105079
Iteration 5, loss = 0.01100820
Iteration 6, loss = 0.01098545
Iteration 7, loss = 0.01097146
Iteration 8, loss = 0.01095536
Iteration 9, loss = 0.01094171
Iteration 10, loss = 0.01092981
Iteration 11, loss = 0.01091858
Iteration 12, loss = 0.01090712
Iteration 13, loss = 0.01089748
Iteration 14, loss = 0.01088757
Iteration 15, loss = 0.01087913
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
[[0.1        0.44444444]
 [0.3        0.38383838]
 [0.5        0.36363636]
 [0.7        0.36363636]
 [0.9        0.37373737]]
Iteration # 1 

Iteration 1, loss = 0.01382360
Iteration 2, loss = 0.00949384
Iteration 3, loss = 0.00837712
Iteration 4, loss = 0.00831913
Iteration 5, loss = 0.00828788
Iteration 6, loss = 0.00825983
Iteration 7, loss = 0.00823914
Iteration 8, loss = 0.00821786
Iteration 9, loss = 0.00819873
It

Iteration 10, loss = 0.00853953
Iteration 11, loss = 0.00851251
Iteration 12, loss = 0.00848978
Iteration 13, loss = 0.00846744
Iteration 14, loss = 0.00844768
Iteration 15, loss = 0.00842958
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
[[0.1        0.57575758]
 [0.3        0.51515152]
 [0.5        0.47474747]
 [0.7        0.44444444]
 [0.9        0.43434343]]
Iteration # 12 

Iteration 1, loss = 0.01148460
Iteration 2, loss = 0.01038079
Iteration 3, loss = 0.00880994
Iteration 4, loss = 0.00824221
Iteration 5, loss = 0.00818187
Iteration 6, loss = 0.00814796
Iteration 7, loss = 0.00812088
Iteration 8, loss = 0.00809694
Iteration 9, loss = 0.00807297
Iteration 10, loss = 0.00805427
Iteration 11, loss = 0.00803664
Iteration 12, loss = 0.00801963
Iteration 13, loss = 0.00800463
Iteration 14, loss = 0.00798990
Iteration 15, loss = 0.00797763
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
[[0.1        0

In [16]:
np.round(vector_opt_bids,3)

array([[0.1  , 0.444, 0.535, 0.576, 0.768, 0.667, 0.646, 0.556, 0.545,
        0.626, 0.636, 0.505, 0.576, 0.566, 0.   , 0.586, 0.545, 0.   ,
        0.556, 0.697, 0.596],
       [0.3  , 0.384, 0.475, 0.505, 0.556, 0.556, 0.545, 0.515, 0.505,
        0.525, 0.545, 0.475, 0.515, 0.505, 0.545, 0.515, 0.495, 0.556,
        0.515, 0.576, 0.556],
       [0.5  , 0.364, 0.434, 0.475, 0.495, 0.495, 0.505, 0.485, 0.485,
        0.475, 0.485, 0.455, 0.475, 0.475, 0.495, 0.485, 0.465, 0.495,
        0.485, 0.505, 0.525],
       [0.7  , 0.364, 0.424, 0.455, 0.475, 0.475, 0.475, 0.465, 0.475,
        0.455, 0.455, 0.444, 0.444, 0.455, 0.465, 0.465, 0.455, 0.465,
        0.465, 0.475, 0.495],
       [0.9  , 0.374, 0.414, 0.444, 0.455, 0.455, 0.465, 0.455, 0.465,
        0.444, 0.444, 0.455, 0.434, 0.444, 0.444, 0.455, 0.444, 0.444,
        0.444, 0.455, 0.465]])

In [18]:
np.savetxt("sealed_bid.csv", vector_opt_bids, delimiter=",")

### Exponential distribution

In [22]:
def prepare_data(n, model=None, start=False, n_bids=100):
    '''Prepare X and y for the neural network'''
    
    # valuations and actions of the player
    val_player   = np.random.exponential(0.5,n)
    action_player  = np.random.exponential(0.5,n)
    val_opponent = np.random.exponential(0.5,n)
    
    # if first iteration - opponent plays randomly
    if start==True:
        action_opponent = np.random.exponential(0.5,n)
        opponent = np.concatenate((val_opponent.reshape(n,1), action_opponent.reshape(n,1)), axis=1)
    else:
        # opponent chooses optimal bid given his valuation
        val_opponent = np.repeat(val_opponent, n_bids)
        action_opponent = np.tile(np.linspace(start=0, stop=1, num=n_bids), n)
        
        # gather private values and bids together
        opponent = np.concatenate((val_opponent.reshape(n*n_bids,1), action_opponent.reshape(n*n_bids,1)), axis=1)
        
        # determine optimal bid
        opponent = opt_bid(model, opponent)
    
    # outcome of the auction
    y = calculate_result(val_player, opponent[:,0], action_player, opponent[:,1])
    
    X = np.concatenate((val_player.reshape(n,1), action_player.reshape(n,1)), axis=1)
    y = np.reshape(y ,[n,]) 
    
    return X, y


def opt_bid(model, X):
    '''Find bid with the highest expected profit'''
    
    # predict expected profit for each pair
    cand_predict = model.predict(X)
    cand_predict[cand_predict <= 0] = 0

    # add expected profit to cand
    cand_res = np.concatenate((X, cand_predict.reshape(X.shape[0],1)), axis=1)

    # sort cand_res in descending order by the expected profit
    cand_res = cand_res[cand_res[:,2].argsort()[::-1],:]

    # keep only best bids
    unique_keys, indices = np.unique(cand_res[:,0], return_index=True)
    cand_res = cand_res[indices,:]
    
    
    return cand_res

In [23]:
np.random.seed(4)

# number of observations in the dataset
n = 1000000

vector_opt_bids = np.array([0.1, 0.3, 0.5, 0.7, 0.9]).reshape(5,1)

for iter in range(20):
    
    print("Iteration #", iter, "\n")
    
    # prepare data
    if iter==0:
        X, y = prepare_data(n, start=True)
    else:
        X, y = prepare_data(n, model=mlp)
        # save this model
        mlp_prev = mlp
        
    # fit the model
    mlp = fit_model(X, y)
    
    # print optimal bids
    vector_opt_bids = np.concatenate((vector_opt_bids, track_progress(mlp)[:,1].reshape(5,1)), axis=1)
    print(vector_opt_bids[:,[0,-1]])
    
    # if there are extreme values in vector_opt_bids - repeat the iteration
    if (np.sum(vector_opt_bids[:,-1]==0) + np.sum(vector_opt_bids[:,-1]==1))!=0:
        print("Repeating iteration")
        mlp = mlp_prev

    del X,y

Iteration # 0 

Iteration 1, loss = 0.05054755
Iteration 2, loss = 0.03422318
Iteration 3, loss = 0.02852051
Iteration 4, loss = 0.02612119
Iteration 5, loss = 0.02491992
Iteration 6, loss = 0.02426299
Iteration 7, loss = 0.02381059
Iteration 8, loss = 0.02336901
Iteration 9, loss = 0.02289992
Iteration 10, loss = 0.02249168
Iteration 11, loss = 0.02215323
Iteration 12, loss = 0.02189557
Iteration 13, loss = 0.02173462
Iteration 14, loss = 0.02161906
Iteration 15, loss = 0.02153530
Iteration 16, loss = 0.02148089
Iteration 17, loss = 0.02142071
Iteration 18, loss = 0.02138722
Iteration 19, loss = 0.02134820
Iteration 20, loss = 0.02131278
Iteration 21, loss = 0.02128682
Iteration 22, loss = 0.02126383
Iteration 23, loss = 0.02123456
Iteration 24, loss = 0.02121000
Iteration 25, loss = 0.02119050
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
[[0.1        0.61616162]
 [0.3        0.26262626]
 [0.5        0.26262626]
 [0.7        0.26262626]
 [0

Iteration 14, loss = 0.01612917
Iteration 15, loss = 0.01607698
Iteration 16, loss = 0.01603161
Iteration 17, loss = 0.01597514
Iteration 18, loss = 0.01593324
Iteration 19, loss = 0.01588522
Iteration 20, loss = 0.01586642
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
[[0.1        0.31313131]
 [0.3        0.31313131]
 [0.5        0.3030303 ]
 [0.7        0.29292929]
 [0.9        0.28282828]]
Iteration # 9 

Iteration 1, loss = 0.05088905
Iteration 2, loss = 0.04124747
Iteration 3, loss = 0.03054841
Iteration 4, loss = 0.02003253
Iteration 5, loss = 0.01815154
Iteration 6, loss = 0.01710834
Iteration 7, loss = 0.01657499
Iteration 8, loss = 0.01631714
Iteration 9, loss = 0.01618273
Iteration 10, loss = 0.01608558
Iteration 11, loss = 0.01602627
Iteration 12, loss = 0.01596695
Iteration 13, loss = 0.01592140
Iteration 14, loss = 0.01587799
Iteration 15, loss = 0.01583130
Iteration 16, loss = 0.01578688
Iteration 17, loss = 0.01574860
Iteration

Iteration 20, loss = 0.01474765
Iteration 21, loss = 0.01472781
Iteration 22, loss = 0.01471178
Iteration 23, loss = 0.01469132
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
[[0.1        0.        ]
 [0.3        0.45454545]
 [0.5        0.39393939]
 [0.7        0.35353535]
 [0.9        0.33333333]]
Repeating iteration
Iteration # 18 

Iteration 1, loss = 0.04517453
Iteration 2, loss = 0.04026778
Iteration 3, loss = 0.02876071
Iteration 4, loss = 0.02104291
Iteration 5, loss = 0.01859585
Iteration 6, loss = 0.01728312
Iteration 7, loss = 0.01668143
Iteration 8, loss = 0.01634799
Iteration 9, loss = 0.01611809
Iteration 10, loss = 0.01594593
Iteration 11, loss = 0.01580088
Iteration 12, loss = 0.01568105
Iteration 13, loss = 0.01558583
Iteration 14, loss = 0.01549966
Iteration 15, loss = 0.01542036
Iteration 16, loss = 0.01534277
Iteration 17, loss = 0.01527785
Iteration 18, loss = 0.01522776
Iteration 19, loss = 0.01517564
Iteration 20, loss =

In [24]:
vector_opt_bids

array([[0.1       , 0.61616162, 0.57575758, 0.        , 0.        ,
        0.57575758, 0.39393939, 0.        , 0.34343434, 0.31313131,
        0.3030303 , 0.88888889, 0.41414141, 0.53535354, 0.64646465,
        0.88888889, 0.67676768, 0.67676768, 0.        , 0.42424242,
        0.47474747],
       [0.3       , 0.26262626, 0.47474747, 0.41414141, 0.35353535,
        0.50505051, 0.39393939, 0.34343434, 0.33333333, 0.31313131,
        0.3030303 , 0.27272727, 0.37373737, 0.45454545, 0.47474747,
        0.52525253, 0.53535354, 0.52525253, 0.45454545, 0.45454545,
        0.45454545],
       [0.5       , 0.26262626, 0.37373737, 0.36363636, 0.33333333,
        0.43434343, 0.37373737, 0.33333333, 0.33333333, 0.3030303 ,
        0.29292929, 0.32323232, 0.33333333, 0.38383838, 0.36363636,
        0.44444444, 0.43434343, 0.43434343, 0.39393939, 0.4040404 ,
        0.41414141],
       [0.7       , 0.26262626, 0.3030303 , 0.32323232, 0.31313131,
        0.37373737, 0.35353535, 0.31313131, 0.3131313

In [25]:
np.savetxt("sealed_bid_exp.csv", vector_opt_bids, delimiter=",")